## [``dyce``](https://posita.github.io/dyce/) translation of “[How do I implement this specialized roll-and-keep mechanic in AnyDice?](https://rpg.stackexchange.com/a/190806)”

Select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import P

def roll_and_keep(p: P, k: int):
    assert p.is_homogeneous()
    max_d = max(p[-1]) if p else 0

    for roll, count in p.rolls_with_counts():
        total = sum(roll[-k:]) + sum(1 for outcome in roll[:-k] if outcome == max_d)
        yield total, count

In [3]:
from dyce import H
from anydyce import jupyter_visualize

d = 6
keep = 3
iterations = 8

def normal():
    for n in range(keep + 1, keep + 1 + iterations):
        p = n @ P(d)
        yield f"{n}d{d} keep {keep}", p.h(slice(-keep, None))

def roll_and_keep_hs():
    for n in range(keep + 1, keep + 1 + iterations):
        p = n @ P(d)
        yield f"{n}d{d} keep {keep} add +1", H(roll_and_keep(p, keep))

jupyter_visualize(
    (item for pair in zip(normal(), roll_and_keep_hs()) for item in pair),
    initial_burst_zero_fill_normalize=True,
    initial_markers="o.",
    selected_name="Line Plot",
)